# **Threads - Escalonamento e Sincronismo** 

### **Autoras** 👩‍💻

Desenvolvedores que contribuíram para a estruturação e desenvolvimento deste projeto:

- **Iza Lopes Ribeiro** - [Izalp](https://github.com/Izalp)
- **Wiliane Carolina Silva** - [wilicarol](https://github.com/wilicarol)

## **Simulação das modalidades**

In [2]:
import threading
import time
import random
from sports import sports


# Função para simular a execução do esporte
def sport(title, duration, lock, wait_times):
    start_time = time.perf_counter()  
    print(f'Modalidade {title} | Início: {start_time:.3f} segundos')

    time.sleep(duration)  # Simula a execução do esporte pelo tempo de duração

    end_time = time.perf_counter() 
    wait_time = end_time - start_time  # Calcula o tempo de espera

    with lock:
        wait_times.append(wait_time)

    print(f'Modalidade {title} | Término: {end_time:.3f} segundos | Duração: {wait_time:.3f} segundos')

## **Escalonamento FCFS (First-Come, First-Served)**

In [3]:
def escalonamento_fcfs(pending_sports, lock, wait_times):
    while pending_sports:
        title, duration = pending_sports.pop(0)
        t = threading.Thread(target=sport, args=(title, duration, lock, wait_times))
        t.start()
        t.join()  # Executa o esporte até completar

In [4]:
def start_olympics_fcfs():
    print('\nOs Jogos Olímpicos com FCFS foram iniciados!\n')

    all_sports = sports.copy()  # Copia dos esportes disponíveis
    pending_sports = []  # Lista de esportes que não puderam ser executados

    lock = threading.Lock() # Bloqueio para que apenas uma thread acesse a seção crítica por vez
    wait_times = []  # Lista para armazenar os tempos de espera

    # Adiciona esportes ao dia conforme há tempo disponível
    while all_sports:
        title = random.choice(all_sports)
        all_sports.remove(title)
        duration = random.uniform((1/60), 5)  # Durações aleatórias

        pending_sports.append((title, duration))

    escalonamento_fcfs(pending_sports, lock, wait_times)

    # Calcula a média de espera
    avg_wait_time = sum(wait_times) / len(wait_times) if wait_times else 0
    print(f'\nMédia de tempo de espera (FCFS): {avg_wait_time:.3f} segundos\n')

start_olympics_fcfs()


Os Jogos Olímpicos com FCFS foram iniciados!

Modalidade Basquete | Início: 2170534.198 segundos
Modalidade Basquete | Término: 2170535.942 segundos | Duração: 1.744 segundos
Modalidade Natação | Início: 2170535.951 segundos
Modalidade Natação | Término: 2170537.447 segundos | Duração: 1.497 segundos
Modalidade Surfe | Início: 2170537.454 segundos
Modalidade Surfe | Término: 2170541.552 segundos | Duração: 4.098 segundos
Modalidade Ginástica Artística | Início: 2170541.560 segundos
Modalidade Ginástica Artística | Término: 2170543.959 segundos | Duração: 2.399 segundos
Modalidade Esgrima | Início: 2170543.967 segundos
Modalidade Esgrima | Término: 2170547.784 segundos | Duração: 3.817 segundos
Modalidade Canoagem Slalom | Início: 2170547.790 segundos
Modalidade Canoagem Slalom | Término: 2170550.029 segundos | Duração: 2.239 segundos
Modalidade Ciclismo de Estrada | Início: 2170550.036 segundos
Modalidade Ciclismo de Estrada | Término: 2170552.298 segundos | Duração: 2.262 segundos
Mo

## **Escalonamento Round Robin (RR)**

In [5]:
def escalonamento_rr(pending_sports, lock, wait_times, quantum=2):
    while pending_sports:
        title, duration = pending_sports.pop(0)
        t = threading.Thread(target=sport, args=(title, min(duration, quantum), lock, wait_times))
        t.start()
        t.join()

        # Se o esporte não terminou, retorna com o tempo restante
        if duration > quantum:
            pending_sports.append((title, duration - quantum))

In [6]:
def start_olympics_rr():
    print('\nOs Jogos Olímpicos com RR foram iniciados!\n')

    all_sports = sports.copy()  # Copia dos esportes disponíveis
    pending_sports = []  # Lista de esportes que não puderam ser executados

    lock = threading.Lock()
    wait_times = []  # Lista para armazenar os tempos de espera

    # Adiciona esportes ao dia conforme há tempo disponível
    while all_sports:
        title = random.choice(all_sports)
        all_sports.remove(title)
        duration = random.uniform((1/60), 5) # Durações aleatórias

        pending_sports.append((title, duration))

    escalonamento_rr(pending_sports, lock, wait_times)

    # Calcula a média de espera
    avg_wait_time = sum(wait_times) / len(wait_times) if wait_times else 0
    print(f'\nMédia de tempo de espera (RR): {avg_wait_time:.3f} segundos\n')

# Inicia a simulação com Round Robin
start_olympics_rr()


Os Jogos Olímpicos com RR foram iniciados!

Modalidade Breaking | Início: 2170670.368 segundos
Modalidade Breaking | Término: 2170672.369 segundos | Duração: 2.001 segundos
Modalidade Pentatlo Moderno | Início: 2170672.377 segundos
Modalidade Pentatlo Moderno | Término: 2170674.378 segundos | Duração: 2.000 segundos
Modalidade Levantamento de Peso | Início: 2170674.382 segundos
Modalidade Levantamento de Peso | Término: 2170674.885 segundos | Duração: 0.504 segundos
Modalidade Handebol | Início: 2170674.888 segundos
Modalidade Handebol | Término: 2170676.115 segundos | Duração: 1.226 segundos
Modalidade Rugby Sevens | Início: 2170676.122 segundos
Modalidade Rugby Sevens | Término: 2170677.477 segundos | Duração: 1.355 segundos
Modalidade Ginástica Artística | Início: 2170677.486 segundos
Modalidade Ginástica Artística | Término: 2170679.099 segundos | Duração: 1.613 segundos
Modalidade Polo Aquático | Início: 2170679.104 segundos
Modalidade Polo Aquático | Término: 2170681.105 segundo

## **Implementação dos Mecanismos de Sincronização**

###  Semáforo Binário

In [7]:
binary_semaphore = threading.Semaphore(1)

def sport_with_binary_semaphore(title, duration, daily_total_hours):
    print(f'[Semáforo Binário] Modalidade {title} aguardando acesso.')
    with binary_semaphore:
        print(f'[Semáforo Binário] Modalidade {title} iniciou execução.')
        sport(title, duration, daily_total_hours, threading.Lock())
    print(f'[Semáforo Binário] Modalidade {title} liberou o acesso.')


### Monitor

In [9]:
monitor_lock = threading.Lock()

def sport_with_monitor(title, duration, daily_total_hours):
    print(f'[Monitor] Modalidade {title} aguardando acesso.')
    with monitor_lock:
        print(f'[Monitor] Modalidade {title} iniciou execução.')
        sport(title, duration, daily_total_hours, threading.Lock())
    print(f'[Monitor] Modalidade {title} liberou o acesso.')


### Semáforo de Contagem

In [10]:
counting_semaphore = threading.BoundedSemaphore(2)  # Permite até 2 threads simultâneas

def sport_with_counting_semaphore(title, duration, daily_total_hours):
    print(f'[Semáforo de Contagem] Modalidade {title} aguardando acesso.')
    with counting_semaphore:
        print(f'[Semáforo de Contagem] Modalidade {title} iniciou execução.')
        sport(title, duration, daily_total_hours, threading.Lock())
    print(f'[Semáforo de Contagem] Modalidade {title} liberou o acesso.')


## **Escalonamento com Sincronismo**

### FCFS + Semáforo Binário

In [11]:
lock = threading.Lock()
binary_semaphore = threading.Semaphore(1)

def sport_with_binary_semaphore(title, duration, daily_total_hours, wait_times):
    print(f'[Semáforo Binário] Modalidade {title} aguardando acesso.')
    start_wait_time = time.perf_counter() 

    with binary_semaphore:
        print(f'[Semáforo Binário] Modalidade {title} iniciou execução.')
        time.sleep(duration)  # Simula o tempo de execução do esporte
        with lock:
            daily_total_hours[0] += duration 

    end_wait_time = time.perf_counter()  
    wait_time = end_wait_time - start_wait_time  # Calcula o tempo de espera total
    
    with lock:
        wait_times.append(wait_time) 
    print(f'[Semáforo Binário] Modalidade {title} liberou o acesso.')

In [12]:
# Escalonamento FCFS com semáforo binário
def fcfs_with_binary_semaphore(pending_sports, daily_total_hours, wait_times):
    executed_today = []
    while pending_sports:
        title, duration = pending_sports.pop(0)
        t = threading.Thread(target=sport_with_binary_semaphore, args=(title, duration, daily_total_hours, wait_times))
        executed_today.append((t, title))
        t.start()
        t.join()  # FCFS não permite interrupção, espera até terminar antes de iniciar o próximo

    return executed_today

In [13]:
def start_fcfs_binary_semaphore():
    print('\nOs Jogos Olímpicos com FCFS foram iniciados!\n')

    all_sports = sports.copy()  # Copia dos esportes disponíveis
    pending_sports = []  # Lista de esportes que não puderam ser executados
    daily_total_hours = [0]  # Lista para manter o total de horas do dia
    wait_times = []  # Lista para armazenar os tempos de espera

    while all_sports:
        title = random.choice(all_sports)
        all_sports.remove(title)
        duration = random.uniform((1/60), 5)  # Durações aleatórias

        pending_sports.append((title, duration))

    fcfs_with_binary_semaphore(pending_sports, daily_total_hours, wait_times)

    # Calcula a média de espera
    avg_wait_time = sum(wait_times) / len(wait_times) if wait_times else 0
    print(f'\nMédia de tempo de espera (FCFS + Semáforo Binário): {avg_wait_time:.3f} segundos\n')


# Inicia a simulação com FCFS e semáforo binário
start_fcfs_binary_semaphore()


Os Jogos Olímpicos com FCFS foram iniciados!

[Semáforo Binário] Modalidade Breaking aguardando acesso.
[Semáforo Binário] Modalidade Breaking iniciou execução.
[Semáforo Binário] Modalidade Breaking liberou o acesso.
[Semáforo Binário] Modalidade Tiro com Arco aguardando acesso.
[Semáforo Binário] Modalidade Tiro com Arco iniciou execução.
[Semáforo Binário] Modalidade Tiro com Arco liberou o acesso.
[Semáforo Binário] Modalidade Ginástica Rítmica aguardando acesso.
[Semáforo Binário] Modalidade Ginástica Rítmica iniciou execução.
[Semáforo Binário] Modalidade Ginástica Rítmica liberou o acesso.
[Semáforo Binário] Modalidade Rugby Sevens aguardando acesso.
[Semáforo Binário] Modalidade Rugby Sevens iniciou execução.
[Semáforo Binário] Modalidade Rugby Sevens liberou o acesso.
[Semáforo Binário] Modalidade Ciclismo Mountain Bike aguardando acesso.
[Semáforo Binário] Modalidade Ciclismo Mountain Bike iniciou execução.
[Semáforo Binário] Modalidade Ciclismo Mountain Bike liberou o acess

### RR + Semáforo de Contagem

In [14]:
lock = threading.Lock()
counting_semaphore = threading.BoundedSemaphore(2)  # Permite até 2 threads simultâneas

def sport_with_counting_semaphore(title, duration, daily_total_hours, wait_times):
    print(f'[Semáforo de Contagem] Modalidade {title} aguardando acesso.')
    start_wait_time = time.perf_counter() 

    with counting_semaphore:
        print(f'[Semáforo de Contagem] Modalidade {title} iniciou execução.')
        time.sleep(duration)  # Simula o tempo de execução do esporte
        with lock:
            daily_total_hours[0] += duration 

    end_wait_time = time.perf_counter()  
    wait_time = end_wait_time - start_wait_time  # Calcula o tempo de espera total
    
    with lock:
        wait_times.append(wait_time) 
    print(f'[Semáforo de Contagem] Modalidade {title} liberou o acesso.')

In [15]:
# Escalonamento RR com semáforo de contagem
def rr_with_counting_semaphore(pending_sports, daily_total_hours, wait_times, quantum=2):
    executed_today = []
    while pending_sports:
        title, duration = pending_sports.pop(0)
        t = threading.Thread(target=sport_with_counting_semaphore, args=(title, min(duration, quantum), daily_total_hours, wait_times))
        executed_today.append((t, title))

        t.start()
        t.join()

        # Se o esporte não terminou, retorna com o tempo restante
        if duration > quantum:
            pending_sports.append((title, duration - quantum))

    return executed_today

In [16]:
def start_rr_counting_semaphore():
    print('\nOs Jogos Olímpicos com RR foram iniciados!\n')

    all_sports = sports.copy()  # Copia dos esportes disponíveis
    pending_sports = []  # Lista de esportes que não puderam ser executados no dia anterior
    daily_total_hours = [0]  # Lista para manter o total de horas do dia
    wait_times = []  # Lista para armazenar os tempos de espera

    while all_sports:
        title = random.choice(all_sports)
        all_sports.remove(title)
        duration = random.uniform((1/60), 5)  # Durações aleatórias

        pending_sports.append((title, duration))

    rr_with_counting_semaphore(pending_sports, daily_total_hours, wait_times)

    # Calcula a média de espera
    avg_wait_time = sum(wait_times) / len(wait_times) if wait_times else 0
    print(f'\nMédia de tempo de espera (RR + Semáforo de Contagem): {avg_wait_time:.3f} segundos\n')    


# Inicia a simulação com RR e semáforo de contagem
start_rr_counting_semaphore()



Os Jogos Olímpicos com RR foram iniciados!

[Semáforo de Contagem] Modalidade Tiro Esportivo aguardando acesso.
[Semáforo de Contagem] Modalidade Tiro Esportivo iniciou execução.
[Semáforo de Contagem] Modalidade Tiro Esportivo liberou o acesso.
[Semáforo de Contagem] Modalidade Vôlei aguardando acesso.
[Semáforo de Contagem] Modalidade Vôlei iniciou execução.
[Semáforo de Contagem] Modalidade Vôlei liberou o acesso.
[Semáforo de Contagem] Modalidade Tênis de Mesa aguardando acesso.
[Semáforo de Contagem] Modalidade Tênis de Mesa iniciou execução.
[Semáforo de Contagem] Modalidade Tênis de Mesa liberou o acesso.
[Semáforo de Contagem] Modalidade Nado Artístico aguardando acesso.
[Semáforo de Contagem] Modalidade Nado Artístico iniciou execução.
[Semáforo de Contagem] Modalidade Nado Artístico liberou o acesso.
[Semáforo de Contagem] Modalidade Esgrima aguardando acesso.
[Semáforo de Contagem] Modalidade Esgrima iniciou execução.
[Semáforo de Contagem] Modalidade Esgrima liberou o aces

### RR + Monitor

In [20]:
monitor_lock = threading.Lock()
num_sports = 5            # número de threads simultâneas para sincronizar
start_barrier = threading.Barrier(num_sports)

def sport_with_monitor(title, duration, daily_total_hours, wait_times):
    print(f'[Monitor] Modalidade {title} aguardando sincronismo.')
    start_wait_time = time.perf_counter() 

    with monitor_lock:
        print(f'[Monitor] Modalidade {title} iniciou execução.')
        time.sleep(duration)  # Simula o tempo de execução do esporte
        with lock:
            daily_total_hours[0] += duration

    end_wait_time = time.perf_counter()  
    wait_time = end_wait_time - start_wait_time  # Calcula o tempo de espera total
    
    with lock:
        wait_times.append(wait_time) 
    print(f'[Monitor] Modalidade {title} liberou o acesso.')

In [21]:
# Escalonamento RR com monitor
def rr_with_monitor(pending_sports, daily_total_hours, wait_times, quantum=2):
    executed_today = []
    while pending_sports:
        title, duration = pending_sports.pop(0)
        t = threading.Thread(target=sport_with_monitor, args=(title, min(duration, quantum), daily_total_hours, wait_times))
        executed_today.append((t, title))

        t.start()
        t.join()

        # Se o esporte não terminou, retorna com o tempo restante
        if duration > quantum:
            pending_sports.append((title, duration - quantum))

    return executed_today

In [22]:
def start_rr_monitor():
    print('\nOs Jogos Olímpicos com RR foram iniciados!\n')

    all_sports = sports.copy()  # Copia dos esportes disponíveis
    pending_sports = []  # Lista de esportes que não puderam ser executados no dia anterior
    daily_total_hours = [0]  # Lista para manter o total de horas do dia
    wait_times = []  # Lista para armazenar os tempos de espera

    while all_sports:
        title = random.choice(all_sports)
        all_sports.remove(title)
        duration = random.uniform((1/60), 5)  # Durações aleatórias

        pending_sports.append((title, duration))

    rr_with_monitor(pending_sports, daily_total_hours, wait_times)

    # Calcula a média de espera
    avg_wait_time = sum(wait_times) / len(wait_times) if wait_times else 0
    print(f'\nMédia de tempo de espera (RR + Monitor): {avg_wait_time:.3f} segundos\n')    


# Inicia a simulação com RR e monitor
start_rr_monitor()


Os Jogos Olímpicos com RR foram iniciados!

[Monitor] Modalidade Boxe aguardando sincronismo.
[Monitor] Modalidade Boxe iniciou execução.
[Monitor] Modalidade Boxe liberou o acesso.
[Monitor] Modalidade Vôlei de Praia aguardando sincronismo.
[Monitor] Modalidade Vôlei de Praia iniciou execução.
[Monitor] Modalidade Vôlei de Praia liberou o acesso.
[Monitor] Modalidade Tiro Esportivo aguardando sincronismo.
[Monitor] Modalidade Tiro Esportivo iniciou execução.
[Monitor] Modalidade Tiro Esportivo liberou o acesso.
[Monitor] Modalidade Triatlo aguardando sincronismo.
[Monitor] Modalidade Triatlo iniciou execução.
[Monitor] Modalidade Triatlo liberou o acesso.
[Monitor] Modalidade Vela aguardando sincronismo.
[Monitor] Modalidade Vela iniciou execução.
[Monitor] Modalidade Vela liberou o acesso.
[Monitor] Modalidade Ciclismo de Estrada aguardando sincronismo.
[Monitor] Modalidade Ciclismo de Estrada iniciou execução.
[Monitor] Modalidade Ciclismo de Estrada liberou o acesso.
[Monitor] Mod

#### Comparação de resultado: 

FCFS será mais simples, com as modalidades executando uma por vez. O tempo total de execução será maior, pois nenhuma modalidade é interrompida.

Round Robin com semáforo de contagem permite execuções simultâneas, mas interrompe modalidades que ultrapassam o quantum, o que pode melhorar a eficiência em termos de utilização de recursos.

OBS.: O monitor não foi utilizado diretamente no código anterior porque, na prática, ele é bastante semelhante ao semáforo binário.